In [545]:
import pandas as pd
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary

In [546]:
import getpass

username = getpass.getuser()

In [547]:
fashion_train_filename = f'/Users/{username}/data/fashion_mnist/fashion-mnist_train.csv'
fashion_val_filename = f'/Users/{username}/data/fashion_mnist/fashion-mnist_test.csv'

text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']

In [595]:
dataset[0][1]

2

In [597]:
np.eye(10)[1]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [592]:
F.one_hot(y, num_classes=10)

TypeError: one_hot(): argument 'input' (position 1) must be Tensor, not numpy.int64

In [556]:
import pandas as pd
from PIL import Image
import numpy as np

idx = 0
df = pd.read_csv(fashion_train_filename)
label = df.iloc[idx][0]
img = df.iloc[idx][1:]

img = np.array(img).reshape((28, 28)).astype(np.uint8)
print(img.shape)
img = Image.fromarray(img)

(28, 28)


In [554]:
img.shape

AttributeError: shape

In [553]:
img

In [598]:
from torch.utils.data import Dataset
import numpy as np
import torch
import pandas as pd


class FashionDataset(Dataset):
    def __init__(self, train_filename, img_size=(28, 28), num_classes=10,
                 train_transforms=None):
        self.df = pd.read_csv(train_filename)
        self.img_size = img_size
        self.num_classes = num_classes
        self.train_transforms = train_transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label_idx = self.df.iloc[idx][0]
        label = np.eye(self.num_classes)[label_idx]

        img = self.df.iloc[idx][1:]
        img = np.array(img).reshape(self.img_size).astype(np.uint8)
        img = Image.fromarray(img)

        if self.train_transforms:
            img = self.train_transforms(img)
        return img, label

In [599]:
from yullab.models import MLP
from torchvision import transforms

train_transforms = transforms.Compose([transforms.ToTensor()])

fashion_dataset = FashionDataset(fashion_train_filename,
                                 train_transforms=train_transforms)
dataloader = DataLoader(fashion_dataset, batch_size=32, shuffle=True)

In [586]:
from yullab.models import MLP

lr = 0.001
n_epochs = 30
sequence_len = 28
input_size = 28

optimizer = optim.Adam(rnn.parameters(), lr=lr)
#criterion = nn.MultiLabelSoftMarginLoss() ## 공부할 것
criterion = nn.NLLLoss()

total_batch = len(dataloader)
train_losses = []
for epoch in range(n_epochs):
    total_cost = 0
    for imgs, labels in dataloader:
        optimizer.zero_grad()

        #imgs = imgs.reshape(sequence_len, -1, input_size) #for rnn

        outputs = lstm(imgs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        total_cost += loss.item()
        train_losses.append(loss.item())

    avg_loss = total_cost / len(dataloader)
    print(f'Epoch: {epoch+1} Cost: {avg_loss}')

RuntimeError: input must have 3 dimensions, got 5

In [591]:
x, y = dataset[0]
y

2

In [ ]:
# for cnn, mlp
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(0, 1)])

train_dataset = FashionDataset(fashion_train_filename,
                         train_transforms=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = FashionDataset(fashion_val_filename,
                             train_transforms=train_transforms)
val_loader = DataLoader(val_dataset, batch_size=10000, shuffle=False)
net = MLP()

#for cnn, mlp
lr = 0.005
n_epochs = 30

optimizer = optim.Adam(net.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

total_batch = len(train_loader)
train_losses = []
test_losses = []
for epoch in range(n_epochs):
    running_loss, train_losses = train(train_loader, net, criterion, optimizer)
    test_loss, test_losses, accuracy  = test(val_loader, net, criterion)
    #
    #
    # running_loss = 0
    # for imgs, labels in train_loader:
    #     optimizer.zero_grad()
    #
    #     imgs = imgs.reshape(-1, 784)
    #     outputs = net(imgs)
    #     loss = criterion(outputs, labels)
    #     loss.backward()
    #
    #     optimizer.step()
    #
    #     running_loss += loss.item()
    #     train_losses.append(loss.item())
    #
    # test_loss = 0
    # accuracy = 0
    # with torch.no_grad():
    #     for images, labels in val_loader:
    #         images = images.reshape(-1, 784)
    #         outputs = net(images)
    #
    #         test_loss += criterion(outputs, labels)
    #
    #         top_p, top_class = outputs.topk(1, dim=1)
    #         _, label_idx = labels.topk(1, dim=1)
    #
    #         equals = top_class == label_idx
    #         accuracy += torch.mean(equals.type(torch.FloatTensor))
    #
    # train_losses.append(running_loss/len(train_loader))
    # test_losses.append(test_loss/len(val_loader))

    print("Epoch: {}/{}.. ".format(epoch+1, n_epochs),
          "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
          "Test Loss: {:.3f}.. ".format(test_loss/len(val_loader)),
          "Test Accuracy: {:.3f}".format(accuracy/len(val_loader)))


Epoch: 1/30..  Training Loss: 1.574..  Test Loss: 1.546..  Test Accuracy: 0.460
Epoch: 2/30..  Training Loss: 1.216..  Test Loss: 1.086..  Test Accuracy: 0.671
Epoch: 3/30..  Training Loss: 1.080..  Test Loss: 1.070..  Test Accuracy: 0.667
Epoch: 4/30..  Training Loss: 1.060..  Test Loss: 1.076..  Test Accuracy: 0.676
Epoch: 5/30..  Training Loss: 1.048..  Test Loss: 1.090..  Test Accuracy: 0.672
Epoch: 6/30..  Training Loss: 1.041..  Test Loss: 1.031..  Test Accuracy: 0.683
Epoch: 7/30..  Training Loss: 0.853..  Test Loss: 0.846..  Test Accuracy: 0.771
Epoch: 8/30..  Training Loss: 0.805..  Test Loss: 0.832..  Test Accuracy: 0.775
Epoch: 9/30..  Training Loss: 0.802..  Test Loss: 0.820..  Test Accuracy: 0.768
Epoch: 10/30..  Training Loss: 0.796..  Test Loss: 0.853..  Test Accuracy: 0.778
Epoch: 11/30..  Training Loss: 0.792..  Test Loss: 0.839..  Test Accuracy: 0.763
Epoch: 12/30..  Training Loss: 0.782..  Test Loss: 0.823..  Test Accuracy: 0.771
Epoch: 13/30..  Training Loss: 0.784.